In [ ]:
!pip install albumentations==1.2.1
!pip install openvino
!pip install segmentation_models_pytorch
!pip install torch-lr-finder
from torch_lr_finder import LRFinder
import segmentation_models_pytorch as smp
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import cv2
from PIL import Image
import numpy as np
import zipfile
import random
import matplotlib
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab.patches import cv2_imshow
import gc
from torch.utils.tensorboard import SummaryWriter

os.environ['PYTHONHASHSEED'] = str(27)
random.seed(27)
np.random.seed(27)
torch.manual_seed(27)
torch.cuda.manual_seed(27)
torch.cuda.manual_seed_all(27)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

drive.mount('/content/drive')
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.8 MB 56.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 376 kB 51.3 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=db00653724a00c02ebd3c220c8b94f7e85a749706f1975f19b550ab01449e7bb
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=f4a62e583cbcb44af339510c40c498e22ca871c9f3b119c3db17d2d98c16b464
  Stored in directo

In [ ]:
def seed_everything(seed=27):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
zipFile = zipfile.ZipFile('/content/drive/MyDrive/Datasets/synthetic.zip', 'r')
zipFile.extractall('dataset')
zipFile.close()

In [ ]:
def get_mean_and_std(data_loader):
  mean = 0
  std = 0
  images_count = 0
  for images, _ in data_loader:
    
    images_in_the_batch = images.size(0)
    images = images.view(images_in_the_batch, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    images_count += images_in_the_batch
  
  mean /= images_count
  std /= images_count
  
  return mean, std

In [ ]:
file = open("/content/dataset/synthetic/lists/train_lst.txt")
text = file.read()
file.close()
arr = text.split('\n')
if '' in arr:
  arr.remove('')
image_path = "/content/dataset/synthetic/images"
masks_path = "/content/dataset/synthetic/masks"
img_path = image_path+"/Img_"+arr[23]+".jpeg"
mask_path = masks_path+"/Mask_"+arr[23]+".png"
image = np.array(Image.open(img_path).convert("RGB"))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
# mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
mask = cv2.resize(mask, (100,100))
# cv2_imshow(image)
mask[mask == 107] = 0
mask[mask == 195] = 1
mask[mask == 88] = 2
mask[mask == 70] = 3
mask[mask == 225] = 4
# np.eye(5, dtype='uint8')[mask]
# cv2_imshow(mask)
print(mask.shape)

# 0 vista = 107
# 1 building = 195
# 2 road = 88
# 3 tree = 70
# 4 cable = 225

(100, 100)


In [ ]:
COLORS = np.array([
    [150,  95,  60], # 0 vista
    [195, 195, 195], # 1 building
    [ 88,  88,  88], # 2 road
    [  0, 120,   0], # 3 tree
    [255, 255,   0], # 4 cable
], dtype=np.uint8)

#Model

In [ ]:
class Conv_block(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)

  def forward(self, inputs):
    x = self.conv1(inputs)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)

    return x

class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # C x H x W -> C x 1 x 1
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class Downsample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.conv = Conv_block(in_channels, out_channels)
    self.pool = nn.MaxPool2d((2, 2))

  def forward(self, inputs):
    x = self.conv(inputs)
    p = self.pool(x)

    return x, p


class Upsample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2, padding=0)
    self.conv = Conv_block(out_channels + out_channels, out_channels)
    self.attention = SqueezeExcitation(out_channels, out_channels//2)

  def forward(self, inputs, skip):
    x = self.up(inputs)
    if x.shape != skip.shape:
      x = TF.resize(x, size=skip.shape[2:])
    x = torch.cat([x, skip], axis=1)
    x = self.conv(x)
    x = self.attention(x)

    return x


class unet(nn.Module):
  def __init__(self, num_classes = 5):
    super().__init__()


    self.down1 = Downsample(3, 64)
    self.down2 = Downsample(64, 128)
    self.down3 = Downsample(128, 256)
    self.down4 = Downsample(256, 512)


    self.transition = Conv_block(512, 1024)


    self.up1 = Upsample(1024, 512)
    self.up2 = Upsample(512, 256)
    self.up3 = Upsample(256, 128)
    self.up4 = Upsample(128, 64)

   
    self.outputs = nn.Conv2d(64, num_classes, kernel_size=1, padding=0)

  def forward(self, inputs):
    skip1, x = self.down1(inputs)
    skip2, x = self.down2(x)
    skip3, x = self.down3(x)
    skip4, x = self.down4(x)


    x = self.transition(x)


    x = self.up1(x, skip4)
    x = self.up2(x, skip3)
    x = self.up3(x, skip2)
    x = self.up4(x, skip1)


    outputs = self.outputs(x)

    return outputs

#Assets

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    jaccard_idx = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.softmax(model(x), 1)
            _, preds = torch.max(preds, dim=1)
            preds = preds.type(torch.float)
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * num_correct +1e-8) / (
                2 * num_correct + num_pixels - num_correct + 1e-8
            )
            jaccard_idx += (num_correct + 1e-8) / (
                num_correct + (preds != y).sum()+ 1e-8
            )
    acc = num_correct/num_pixels
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    dice = dice_score/len(loader)
    print(f"Dice score: {dice_score/len(loader)}")

    jaccard = jaccard_idx/len(loader)
    print(f"Jaccard index: {jaccard}")
    model.train()

    return acc, dice, jaccard

def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

# Datasets and data loaders

In [ ]:
class SyntheticDataset(Dataset):
  def __init__(self, list_path, dir="/content/dataset/synthetic", transform=None ):
    self.file_path = list_path
    self.dir = dir
    self.transforms = transform
    self.images_path = self.dir+"/images"
    self.masks_path = self.dir+"/masks"
    self.file = open(self.file_path, encoding='utf-8')
    self.text = self.file.read()
    self.file.close
    self.image_descriptions = self.text.split("\n")
    if '' in self.image_descriptions:
      self.image_descriptions.remove('')
    

  def __getitem__(self, index):
    img_path = self.images_path+"/Img_"+self.image_descriptions[index]+".jpeg"
    mask_path = self.masks_path+"/Mask_"+self.image_descriptions[index]+".png"
    image = np.array(Image.open(img_path).convert("RGB"))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = np.array(Image.open(mask_path).convert("L"))
    mask[mask == 107] = 0
    mask[mask == 195] = 1
    mask[mask == 88] = 2
    mask[mask == 70] = 3
    mask[mask == 225] = 4
    
    if self.transforms is not None:
      augmentations = self.transforms(image=image, mask=mask)
      image = augmentations["image"]
      mask = augmentations["mask"]
    return image, mask

  
  def __len__(self):
    return len(self.image_descriptions)

In [ ]:
seed_everything()
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 3e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
NUM_EPOCHS = 30
NUM_WORKERS = 2
IMAGE_HEIGHT = 720  # 1080 originally
IMAGE_WIDTH = 1088  # 1920 originally
PIN_MEMORY = True
LOAD_MODEL = False

In [ ]:
seed_everything()
mean = [0.5585, 0.6353, 0.6439]
std = [0.1218, 0.1545, 0.1860]
train_transforms1 = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.9),
            A.VerticalFlip(p=0.9),
            A.Normalize(
                mean=mean,
                std=std,
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

train_transforms2 = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            # A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25),
            # A.Blur(blur_limit=5, p=0.7),
            # A.RandomBrightnessContrast(p=0.5),
            # A.ChannelShuffle(p=0.7),
            # A.Solarize(threshold=128, p=0.5),
            # A.GaussNoise(),
            # A.InvertImg(p=0.7),
            A.Normalize(
                mean=mean,
                std=std,
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

train_transforms3 = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            # A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25),
            # A.Blur(blur_limit=5, p=0.7),
            # A.RandomBrightnessContrast(p=0.5),
            # A.ChannelShuffle(p=0.7),
            A.Solarize(threshold=128, p=0.5),
            A.GaussNoise(),
            A.InvertImg(p=0.7),
            A.Normalize(
                mean=mean,
                std=std,
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )
val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=mean,
                std=std,
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

In [ ]:
seed_everything()
train_dataset1 = SyntheticDataset(
        list_path = "/content/dataset/synthetic/lists/train_lst.txt",
        transform=train_transforms1,
    )

train_dataset2 = SyntheticDataset(
        list_path = "/content/dataset/synthetic/lists/train_lst.txt",
        transform=train_transforms2,
    )

train_dataset3 = SyntheticDataset(
        list_path = "/content/dataset/synthetic/lists/train_lst.txt",
        transform=train_transforms3,
    )

train_dataset = torch.utils.data.ConcatDataset([train_dataset1, train_dataset2, train_dataset3])

train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
    )

val_dataset = SyntheticDataset(
        list_path = "/content/dataset/synthetic/lists/val_lst.txt",
        transform=val_transforms,
    )

val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=False,
    )

#Training

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    seed_everything()
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.type(torch.LongTensor)
        targets = targets.to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
        # predictions = model(data)
        # loss = loss_fn(predictions, targets)

        # backward
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
seed_everything()
weights = torch.tensor([0.4, 1.5, 0.7, 0.4, 1.5], device=DEVICE)
model = smp.DeepLabV3Plus(classes=5, encoder_weights="imagenet")
model = model.to(DEVICE)
# loss_fn = nn.CrossEntropyLoss(weight = weights)
loss_fn1 = smp.losses.DiceLoss("multiclass")
loss_fn2 = nn.CrossEntropyLoss(weight = weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# lr_finder = LRFinder(model, optimizer, loss_fn, device="cuda")
# lr_finder.range_test(train_loader, end_lr=1, num_iter=1000)
# lr_finder.plot()

In [ ]:
# 0 vista = 107
# 1 building = 195
# 2 road = 88
# 3 tree = 70
# 4 cable = 225
seed_everything()
scaler = torch.cuda.amp.GradScaler()
writer = SummaryWriter("/content/boards/board")
step = 0
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, LEARNING_RATE, epochs=NUM_EPOCHS, 
                                               steps_per_epoch=len(train_loader))

for epoch in range(NUM_EPOCHS):
  # train_fn(train_loader, model, optimizer, loss_fn, scaler)
  loop = tqdm(train_loader)

  for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)
      targets = targets.type(torch.LongTensor)
      targets = targets.to(device=DEVICE)


      predictions = model(data)
      loss = 0.4*loss_fn1(predictions, targets) + 0.8*loss_fn2(predictions, targets)

      # backward
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()


      sched.step()
      # update tqdm loop
      loop.set_postfix(loss=loss.item())


  val_acc, dice_score, jaccard_idx = check_accuracy(val_loader, model, device=DEVICE)
  if dice_score > 0.9815:
    torch.save(model, f"Synthetic_sementation_deeplabv3plus_val_acc={val_acc}_dice={dice_score}_Jaccard={jaccard_idx}")


  writer.add_scalar("Validation accuracy", val_acc, global_step=step)
  writer.add_scalar("Dice score", dice_score, global_step=step)
  writer.add_scalar("Jaccard index", jaccard_idx, global_step=step) 
  step += 1
  
writer.close()

100%|██████████| 75/75 [01:04<00:00,  1.16it/s, loss=1.12]


Got 8716429/12533760 with acc 69.54
Dice score: 0.8142071962356567
Jaccard index: 0.8180963397026062


100%|██████████| 75/75 [01:04<00:00,  1.17it/s, loss=0.68]


Got 11010199/12533760 with acc 87.84
Dice score: 0.9294045567512512
Jaccard index: 0.9295482635498047


100%|██████████| 75/75 [01:04<00:00,  1.17it/s, loss=0.348]


...

Got 12115856/12533760 with acc 96.67
Dice score: 0.9812766909599304
Jaccard index: 0.9809854030609131


100%|██████████| 75/75 [01:04<00:00,  1.16it/s, loss=0.0659]


Got 12116522/12533760 with acc 96.67
Dice score: 0.9813218712806702
Jaccard index: 0.9810270071029663


100%|██████████| 75/75 [01:04<00:00,  1.16it/s, loss=0.0512]


Got 12117896/12533760 with acc 96.68
Dice score: 0.9813653230667114
Jaccard index: 0.9810713529586792


#Stuff

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model = unet().to(DEVICE)
x, y = next(iter(train_loader))
x = x.to(DEVICE)
y = y.to(DEVICE)
preds = torch.softmax(model(x), 1)
_, preds = torch.max(preds, dim=1)
preds = preds.type(torch.float)
correct = (preds == y).sum()
incorrect = (preds != y).sum()
total = torch.numel(preds)
print(correct)
print(incorrect)
print(total)
print(correct+incorrect)
gc.collect()
torch.cuda.empty_cache()

tensor(19217, device='cuda:0')
tensor(98543, device='cuda:0')
117760
tensor(117760, device='cuda:0')


In [ ]:
# model = smp.DeepLabV3Plus(classes=5)
# model = model.to(DEVICE)
model = torch.load("/content/Synthetic_sementation_deeplabv3plus_val_acc=0.9671552777290344_dice=0.9815328121185303_Jaccard=0.9812494516372681")
model.eval()
# img, mask = val_dataset[10]
# img = img.unsqueeze(0)
# img = img.to(DEVICE)
# preds = torch.softmax(model(img), 1)
# _, preds = torch.max(preds, dim=1)
# preds = preds.squeeze(0)
# print(img.shape)
# print(mask.shape)

for i in range(len(val_dataset)):
  img, mask = val_dataset[i]
  mask = mask.to(DEVICE)
  img = img.unsqueeze(0)
  img = img.to(DEVICE)
  preds = torch.softmax(model(img), 1)
  _, preds = torch.max(preds, dim=1)
  preds = preds.squeeze(0)
  mask = mask.cpu().detach().numpy()
  mask[mask == 0] = 107
  mask[mask == 1] = 195
  mask[mask == 2] = 88
  mask[mask == 3] = 70
  mask[mask == 4] = 225

  # cv2.imwrite("/content/drive/MyDrive/Synthetic_masks/ground_truth/"+str(i)+".png", mask)

  preds = preds.cpu().detach().numpy()
  preds[preds == 0] = 107
  preds[preds == 1] = 195
  preds[preds == 2] = 88
  preds[preds == 3] = 70
  preds[preds == 4] = 225
  cv2_imshow(mask)
  cv2_imshow(preds)
  break

  # cv2.imwrite("/content/drive/MyDrive/Synthetic_masks/torch/"+str(i)+".png", preds)




# mask = mask.cpu().detach().numpy()
# mask[mask == 0] = 107
# mask[mask == 1] = 195
# mask[mask == 2] = 88
# mask[mask == 3] = 70
# mask[mask == 4] = 225
# preds = preds.cpu().detach().numpy()
# preds[preds == 0] = 107
# preds[preds == 1] = 195
# preds[preds == 2] = 88
# preds[preds == 3] = 70
# preds[preds == 4] = 225

# cv2_imshow(mask)
# cv2_imshow(preds)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/boards

In [ ]:
import openvino
from openvino.runtime import Core

In [ ]:
# model = torch.load("/content/drive/MyDrive/Models/Synthetic_sementation_unet_val_acc=0.9538319706916809_dice=0.9738907814025879_Jaccard=0.9735795259475708", map_location=torch.device('cpu'))
# model.eval()
model = smp.DeepLabV3Plus(classes=5)
batch = torch.randn(1, 3, 720, 1088)

# torch.save(model.state_dict(), f"Syntheti_deeplabv3plus_val_acc=0,9788")

model = torch.load("/content/drive/MyDrive/Models/Synthetic_sementation_deeplabv3plus_val_acc=0.9668765068054199_dice=0.981420636177063_Jaccard=0.9811235666275024", map_location=torch.device('cpu'))

torch.onnx.export(model, batch, "Synthetic_sementation_deeplabv3plus_val_acc=0.9668765068054199_dice=0.981420636177063_Jaccard=0.9811235666275024.onnx", export_params=True, opset_version=11)


ie = Core()
onnx_model = ie.read_model(model="Synthetic_sementation_deeplabv3plus_val_acc=0.9668765068054199_dice=0.981420636177063_Jaccard=0.9811235666275024.onnx")
compiled_onnx_model = ie.compile_model(model=onnx_model, device_name="CPU")
input_layer = compiled_onnx_model.input(0)
output_layer = compiled_onnx_model.output(0)

input_layer = compiled_onnx_model.input(0)
output_layer = compiled_onnx_model.output(0)

num_correct = 0
num_pixels = 0
dice_score = 0
jaccard_idx = 0

for i in range(len(val_dataset)):
  img, mask = val_dataset[i]

  
  img = img.unsqueeze(0)
  preds = compiled_onnx_model([img])[output_layer]
  preds = torch.from_numpy(preds)
  preds = torch.softmax(preds, 1)
  _, preds = torch.max(preds, dim=1)

  preds = preds.type(torch.float)
  num_correct += (preds == mask).sum()
  num_pixels += torch.numel(preds)
  dice_score += (2 * num_correct +1e-8) / (
                2 * num_correct + num_pixels - num_correct + 1e-8
            )
  jaccard_idx += (num_correct + 1e-8) / (
                num_correct + (preds != mask).sum()+ 1e-8
            )
print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
print(f"Dice score: {dice_score/len(val_dataset)}")

jaccard = jaccard_idx/len(val_dataset)
print(f"Jaccard index: {jaccard}")


  # preds = preds.squeeze(0)
  # preds = preds.cpu().detach().numpy()
  # preds[preds == 0] = 107
  # preds[preds == 1] = 195
  # preds[preds == 2] = 88
  # preds[preds == 3] = 70
  # preds[preds == 4] = 225

  # cv2.imwrite("/content/drive/MyDrive/Synthetic_masks/onnx/"+str(i)+".png", preds)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


Got 12118598/12533760 with acc 96.69
Dice score: 0.9810609817504883
Jaccard index: 0.9919564723968506
